<a href="https://colab.research.google.com/github/28humaid/book_recomm_system/blob/main/L_bookRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from scipy import stats

In [30]:
books = pd.read_csv('/content/drive/MyDrive/book_recommender/input/Books.csv')
users = pd.read_csv('/content/drive/MyDrive/book_recommender/input/Users.csv')
ratings = pd.read_csv('/content/drive/MyDrive/book_recommender/input/Ratings.csv')

<ipython-input-30-c39b0eb75fb1>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('/content/drive/MyDrive/book_recommender/input/Books.csv')


In [3]:
books.sample(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
219123,0198612435,The Concise Oxford Dictionary: Thumb Index Edi...,R.E. Allen,1990,Oxford University Press,http://images.amazon.com/images/P/0198612435.0...,http://images.amazon.com/images/P/0198612435.0...,http://images.amazon.com/images/P/0198612435.0...
88894,0373222750,"Timewalker (Harlequin Intrigue, No 275)",Aimee Thurlo,1994,Harlequin,http://images.amazon.com/images/P/0373222750.0...,http://images.amazon.com/images/P/0373222750.0...,http://images.amazon.com/images/P/0373222750.0...
39878,0552142131,A Dog Called Demolition,Robert Rankin,1996,"Acacia Press, Inc.",http://images.amazon.com/images/P/0552142131.0...,http://images.amazon.com/images/P/0552142131.0...,http://images.amazon.com/images/P/0552142131.0...


In [4]:
users.sample(3)

,User-ID,Location,Age
158905,158906,"st. cloud, minnesota, usa",59.0
228833,228834,"geelong, victoria, australia",104.0
243861,243862,"midlothian, virginia, usa",NaN


In [5]:
ratings.sample(3)

,User-ID,ISBN,Book-Rating
981914,236172,0345339509,0
68084,14230,0373252579,0
366820,88066,0764107704,6


## check the shape of data

In [6]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271360, 8)
(278858, 3)
(1149780, 3)


## check the null values

In [7]:
books.isnull().sum()

,0
ISBN,0
Book-Title,0
Book-Author,2
Year-Of-Publication,0
Publisher,2
Image-URL-S,0
Image-URL-M,0
Image-URL-L,3


In [8]:
users.isnull().sum()

,0
User-ID,0
Location,0
Age,110762


In [9]:
ratings.isnull().sum()

,0
User-ID,0
ISBN,0
Book-Rating,0


## check duplicate values

In [10]:
print(books.duplicated().sum())
print(users.duplicated().sum())
print(ratings.duplicated().sum())

0
0
0


#Popularity based reccommender system

In [11]:
# highest average rating wali top 50 books dikhayenge
# ek catch ye hai ki...agr ksi book pr srf 2 votes ho..aur dono 10 ho...tb to average rating 10
# hojayegi...pr wo popular books to nhi hui !
# isiliye srf wahi books lenge jinpr 250 se zada votes honge.
#
#
# WO BOOKS JINPR 250 SE ZADA VOTES HAIN,
# USME FIR HIGHEST RATING K HISAB SE DECREASING ORDER MEI ARRANGE KR DENGE,
# FIR USKE BAAD, TOP KI 50 BOOKS LE LENGE..simple!!

In [12]:
ratings.sample(1)

,User-ID,ISBN,Book-Rating
557043,134118,0684862336,8


In [13]:
# isbn unique h, to uske basis pr group bna aur rows count kr le
# newRatings=ratings.groupby('ISBN').count()

#250 se zada vale le lo
# newRatings=ratings[ratings.groupby('ISBN').count()>250]
# newRatings
# NOT LIKE ABOVE WAY....TRY BELOW WAY

# numOfRatings=ratings.groupby('ISBN')['Book-Rating'].count()
# above line of code gives a series with index 'ISBN' and values as count of non-null values
# of 'Book-Rating'
# numOfRatings=numOfRatings[numOfRatings>250]
# ab 250 se zada wali le le
# numOfRatings

In [14]:
# # ek nya column bna le
# ratings['count-ratings']=ratings.groupby('ISBN')['Book-Rating'].transform('count')
# # 'ISBN' k hisab se group bna kr, 'Book-Rating' ko count kr le, ek nya column 'count-ratings' bna kr
# # usme count ki values daal de

# #ab wo rows le lo jinka 'count-ratings' > 250 hai
# new_ratings=ratings[ratings['count-ratings']>250]
# new_ratings

In [15]:
#drop users columns
new_ratings=ratings.drop('User-ID',axis=1)
new_ratings

,ISBN,Book-Rating
0,034545104X,0
1,0155061224,5
2,0446520802,0
3,052165615X,3
4,0521795028,6
...,...,...
1149775,1563526298,9
1149776,0679447156,0
1149777,0515107662,10
1149778,0590442449,10


In [16]:
# Group by 'ISBN' and create two new columns:
# 1. 'Rating_Count' contains the count of 'Book-Rating' values for each 'ISBN'.
# 2. 'Rating_Mean' contains the mean of 'Book-Rating' values for each 'ISBN'.
new_ratings=new_ratings.groupby('ISBN').agg(
    RatingCount=('Book-Rating','count'),
    RatingMean=('Book-Rating','mean')
).reset_index()
new_ratings

,ISBN,RatingCount,RatingMean
0,0330299891,2,3.0
1,0375404120,2,1.5
2,0586045007,1,0.0
3,9022906116,2,3.5
4,9032803328,1,0.0
...,...,...,...
340551,cn113107,1,0.0
340552,ooo7156103,1,7.0
340553,§423350229,1,0.0
340554,´3499128624,1,8.0


In [17]:
# ab wo rows le lo jinka 'RatingCount' > 250 hai
ratings_250_plus=new_ratings[new_ratings['RatingCount']>250]
ratings_250_plus

,ISBN,RatingCount,RatingMean
7635,0060392452,283,4.356890
7745,0060502258,376,3.430851
9637,0060928336,732,3.448087
9745,0060930535,494,3.609312
9880,0060934417,350,3.702857
...,...,...,...
203767,0805063897,267,4.142322
218729,0842329129,297,4.026936
247408,0971880107,2502,1.019584
248173,1400034779,431,3.417633


In [18]:
#sort this in accordance with descending order of 'RatingMean'
famous_books=ratings_250_plus.sort_values(by='RatingMean', ascending=False)

In [19]:
#take top 50 from here
famous_books=famous_books.head(50)
famous_books

,ISBN,RatingCount,RatingMean
98386,043935806X,334,5.571856
54400,0345339681,281,5.007117
105881,0446310786,389,4.920308
144131,059035342X,571,4.900175
97172,0439064872,351,4.729345
83359,0385504209,883,4.652322
47552,0316769487,403,4.635236
197596,0786868716,427,4.543326
54401,0345339703,257,4.505837
47371,0316666343,1295,4.468726


In [20]:
# ab 'famous_books' aur 'books' ka inner join le le
famous_books_names=pd.merge(famous_books, books, how='left', on='ISBN')

# tada! mil gyi popular books....
famous_books_names

,ISBN,RatingCount,RatingMean,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,043935806X,334,5.571856,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...,http://images.amazon.com/images/P/043935806X.0...
1,0345339681,281,5.007117,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...
2,0446310786,389,4.920308,To Kill a Mockingbird,Harper Lee,1988,Little Brown &amp; Company,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...,http://images.amazon.com/images/P/0446310786.0...
3,059035342X,571,4.900175,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,Arthur A. Levine Books,http://images.amazon.com/images/P/059035342X.0...,http://images.amazon.com/images/P/059035342X.0...,http://images.amazon.com/images/P/059035342X.0...
4,0439064872,351,4.729345,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439064872.0...,http://images.amazon.com/images/P/0439064872.0...,http://images.amazon.com/images/P/0439064872.0...
5,0385504209,883,4.652322,The Da Vinci Code,Dan Brown,2003,Doubleday,http://images.amazon.com/images/P/0385504209.0...,http://images.amazon.com/images/P/0385504209.0...,http://images.amazon.com/images/P/0385504209.0...
6,0316769487,403,4.635236,The Catcher in the Rye,J.D. Salinger,1991,"Little, Brown",http://images.amazon.com/images/P/0316769487.0...,http://images.amazon.com/images/P/0316769487.0...,http://images.amazon.com/images/P/0316769487.0...
7,0786868716,427,4.543326,The Five People You Meet in Heaven,Mitch Albom,2003,Hyperion,http://images.amazon.com/images/P/0786868716.0...,http://images.amazon.com/images/P/0786868716.0...,http://images.amazon.com/images/P/0786868716.0...
8,0345339703,257,4.505837,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339703.0...,http://images.amazon.com/images/P/0345339703.0...,http://images.amazon.com/images/P/0345339703.0...
9,0316666343,1295,4.468726,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown",http://images.amazon.com/images/P/0316666343.0...,http://images.amazon.com/images/P/0316666343.0...,http://images.amazon.com/images/P/0316666343.0...


## Collaborative filtering based recommender system

In [22]:
# consider books jinpr atleast 50 users have rated
# consider users jinhone atleast 200 books ko rate kiya hai

In [24]:
ratings.sample(2)

,User-ID,ISBN,Book-Rating
62725,12657,3551745315,4
786609,190279,0399146008,8


In [33]:
ratings['user_rating_count']=ratings.groupby('User-ID')['ISBN'].transform('count')
ratings

,User-ID,ISBN,Book-Rating,user_rating_count
0,276725,034545104X,0,1
1,276726,0155061224,5,1
2,276727,0446520802,0,1
3,276729,052165615X,3,2
4,276729,0521795028,6,2
...,...,...,...,...
1149775,276704,1563526298,9,17
1149776,276706,0679447156,0,1
1149777,276709,0515107662,10,1
1149778,276721,0590442449,10,1


In [37]:
users_with_50plus_ratings=ratings[ratings['user_rating_count']>=50]
users_with_50plus_ratings

,User-ID,ISBN,Book-Rating,user_rating_count
173,276847,0446364193,0,51
174,276847,3257200552,5,51
175,276847,3379015180,0,51
176,276847,3404145909,8,51
177,276847,3404148576,8,51
...,...,...,...,...
1149744,276688,0836236688,10,91
1149745,276688,0892966548,10,91
1149746,276688,1551669315,6,91
1149747,276688,1575660792,7,91
